In [16]:
# create spark session

from pyspark.sql import SparkSession

spark=(
    SparkSession
    .builder
    .appName("Triggers in Spark Streaming")
    .config("spark.streaming.stopGracefullyonshutdown", True)
    .config("spark.jars.packages","org.apache.spark:spark-sql-kafka-0-10_2.12:3.3.0")
    .config("spark.sql.shuffle.partitions",4)
    .master("local[*]")
    .getOrCreate()
    )
    
spark


In [17]:
# create the kafka_df to read form kafka

kafka_df = (
    spark
    .read
    .format("kafka")
    .option("kafka.bootstrap.servers","ed-kafka:29092")
    .option("subscribe","device-data")
    .option("startingoffsets","earliest")
    .load()

)

In [18]:
# view  schema for raw kafka_df

kafka_df.printSchema()

root
 |-- key: binary (nullable = true)
 |-- value: binary (nullable = true)
 |-- topic: string (nullable = true)
 |-- partition: integer (nullable = true)
 |-- offset: long (nullable = true)
 |-- timestamp: timestamp (nullable = true)
 |-- timestampType: integer (nullable = true)



In [19]:
# parse value from binary to string into kafka_json_df

from pyspark.sql.functions import expr

kafka_json_df = kafka_df.withColumn("value",expr("cast(value as string)"))

In [20]:

from pyspark.sql.types import StringType, StructField, StructType, ArrayType,  LongType

json_schema = (
    StructType(
    [StructField('customerId',StringType(), True),
    StructField('data', StructType(
        [StructField('devices',
                    ArrayType(StructType([
                    StructField('deviceId', StringType(), True),
                    StructField('measure', StringType(), True),
                    StructField('status', StringType(), True),
                    StructField('temperature',LongType(), True)
                ]), True), True)
        ]), True),
StructField('eventId', StringType(), True),
StructField('eventoffset', LongType(), True),
StructField('eventPublisher', StringType(), True),
StructField('eventTime', StringType(), True)
])
)



In [21]:

# Apply the schema to payload to read the data

from pyspark.sql.functions import from_json, col

streaming_df = kafka_json_df.withColumn("values_json", from_json(col("value"), json_schema))

In [22]:
# To the schema of the data, place a sample json file and change readStream to read 

streaming_df.printSchema()


root
 |-- key: binary (nullable = true)
 |-- value: string (nullable = true)
 |-- topic: string (nullable = true)
 |-- partition: integer (nullable = true)
 |-- offset: long (nullable = true)
 |-- timestamp: timestamp (nullable = true)
 |-- timestampType: integer (nullable = true)
 |-- values_json: struct (nullable = true)
 |    |-- customerId: string (nullable = true)
 |    |-- data: struct (nullable = true)
 |    |    |-- devices: array (nullable = true)
 |    |    |    |-- element: struct (containsNull = true)
 |    |    |    |    |-- deviceId: string (nullable = true)
 |    |    |    |    |-- measure: string (nullable = true)
 |    |    |    |    |-- status: string (nullable = true)
 |    |    |    |    |-- temperature: long (nullable = true)
 |    |-- eventId: string (nullable = true)
 |    |-- eventoffset: long (nullable = true)
 |    |-- eventPublisher: string (nullable = true)
 |    |-- eventTime: string (nullable = true)



In [23]:
streaming_df = kafka_json_df.withColumn("values_json", from_json(col("value"), json_schema)).selectExpr("values_json.*")

In [24]:
streaming_df.printSchema()


root
 |-- customerId: string (nullable = true)
 |-- data: struct (nullable = true)
 |    |-- devices: array (nullable = true)
 |    |    |-- element: struct (containsNull = true)
 |    |    |    |-- deviceId: string (nullable = true)
 |    |    |    |-- measure: string (nullable = true)
 |    |    |    |-- status: string (nullable = true)
 |    |    |    |-- temperature: long (nullable = true)
 |-- eventId: string (nullable = true)
 |-- eventoffset: long (nullable = true)
 |-- eventPublisher: string (nullable = true)
 |-- eventTime: string (nullable = true)



In [25]:
# Lets explode the data as devices contains List/array of device reading

from pyspark.sql.functions import explode

explode_df = streaming_df.withColumn("data_devices", explode("data.devices"))

explode_df.printSchema()


root
 |-- customerId: string (nullable = true)
 |-- data: struct (nullable = true)
 |    |-- devices: array (nullable = true)
 |    |    |-- element: struct (containsNull = true)
 |    |    |    |-- deviceId: string (nullable = true)
 |    |    |    |-- measure: string (nullable = true)
 |    |    |    |-- status: string (nullable = true)
 |    |    |    |-- temperature: long (nullable = true)
 |-- eventId: string (nullable = true)
 |-- eventoffset: long (nullable = true)
 |-- eventPublisher: string (nullable = true)
 |-- eventTime: string (nullable = true)
 |-- data_devices: struct (nullable = true)
 |    |-- deviceId: string (nullable = true)
 |    |-- measure: string (nullable = true)
 |    |-- status: string (nullable = true)
 |    |-- temperature: long (nullable = true)



In [26]:
# flatten the explode df
from pyspark.sql.functions import col
 
flattened_df = (
    explode_df
    .drop("data")
    .withColumn("deviceId", col("data_devices.deviceId"))
    .withColumn("measure", col("data_devices.measure"))
    .withColumn("status", col("data_devices.status"))
    .withColumn("temperature", col("data_devices.temperature"))
    .drop("data_devices")

)

flattened_df.printSchema()


root
 |-- customerId: string (nullable = true)
 |-- eventId: string (nullable = true)
 |-- eventoffset: long (nullable = true)
 |-- eventPublisher: string (nullable = true)
 |-- eventTime: string (nullable = true)
 |-- deviceId: string (nullable = true)
 |-- measure: string (nullable = true)
 |-- status: string (nullable = true)
 |-- temperature: long (nullable = true)



In [29]:
# Running in once/availableNow and processingTime mode
# write the output to console sink to check the output

(
    flattened_df
    .writeStream
    .format("console")
    .outputMode("append")
    .trigger(once=True)   # or availableNow=True (Spark 3.5+)
    .option("truncate", "false")
    .option("checkpointLocation", "checkpoint_dir_kafka_1")
    .start()
    .awaitTermination()
)




AnalysisException: 'writeStream' can be called only on streaming Dataset/DataFrame

In [ ]:
(
    flattened_df
    .writeStream
    .format("console")
    .outputMode("append")
    .trigger(once=True)   # or availableNow=True (Spark 3.5+)
    .option("truncate", "false")
    .option("checkpointLocation", "checkpoint_dir_kafka_1")
    .start()
    .awaitTermination()
)
